In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Choix des stations météo conservées pour l'étude

In [3]:
#lecture des stations à partir des données fournies par Météo France
stations_Synop=gpd.read_file(r"C:\DATA\meteo\postesSynop_meteoFrance.json")
kargs={"location":[-30,0],"zoom_start":2}
stations_Synop.explore(
    map_kwds={"scrollWheelZoom":False},
    **kargs
)

#Visuellement on note qu' il y a entre 3 et 5 groupes de stations,avec une grosse grappe sur la France métropolitaine


In [4]:
#Informations du dataframe
stations_Synop.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   ID         62 non-null     object  
 1   Nom        62 non-null     object  
 2   Latitude   62 non-null     object  
 3   Longitude  62 non-null     object  
 4   Altitude   62 non-null     object  
 5   geometry   62 non-null     geometry
dtypes: geometry(1), object(5)
memory usage: 3.0+ KB


In [5]:
#Ajout d'une variable de  distance au centre de Paris
#Coordonnées du centre de Paris (données IGN)
lat_Paris=48.8566667
lon_Paris=2.34222222

#Calcul
import geopy.distance as gdist
stations_Synop["distance_Paris"]=np.zeros(stations_Synop.shape[0])
for i in stations_Synop.index:
    lat = stations_Synop['Latitude'].iloc[i]
    lon = stations_Synop['Longitude'].iloc[i]
    stations_Synop["distance_Paris"][i]=gdist.geodesic((lat,lon),(lat_Paris,lon_Paris))
#Mise au format
stations_Synop["distance_Paris"]= stations_Synop["distance_Paris"].apply(lambda x:str(x).split(" ")[0])   
stations_Synop["distance_Paris"]=stations_Synop["distance_Paris"].astype("float64").round(3)

stations_Synop=stations_Synop.rename({"ID":"ID_station","Nom":"Nom_station"},axis=1)
stations_Synop   

,ID_station,Nom_station,Latitude,Longitude,Altitude,geometry,distance_Paris
0,07005,ABBEVILLE,50.136000,1.834000,69,POINT (1.83400 50.13600),146.972
1,07015,LILLE-LESQUIN,50.570000,3.097500,47,POINT (3.09750 50.57000),198.192
2,07020,PTE DE LA HAGUE,49.725167,-1.939833,6,POINT (-1.93983 49.72517),326.075
3,07027,CAEN-CARPIQUET,49.180000,-0.456167,67,POINT (-0.45617 49.18000),207.810
4,07037,ROUEN-BOOS,49.383000,1.181667,151,POINT (1.18167 49.38300),102.969
...,...,...,...,...,...,...,...
57,81401,SAINT LAURENT,5.485500,-54.031667,5,POINT (-54.03167 5.48550),7139.314
58,81405,CAYENNE-MATOURY,4.822333,-52.365333,4,POINT (-52.36533 4.82233),7085.840
59,81408,SAINT GEORGES,3.890667,-51.804667,6,POINT (-51.80467 3.89067),7131.279
60,81415,MARIPASOULA,3.640167,-54.028333,106,POINT (-54.02833 3.64017),7301.339


In [6]:
#Filtre sur la distance à Paris < 1000 km --> on s'intéresse à la température en métropole
stations_Synop=stations_Synop[stations_Synop["distance_Paris"]<1000]

In [7]:
#Carte des stations conservées
kargs={"location":[47,0],"zoom_start":5}
stations_Synop.explore(
    marker_kwds={"radius":5,"fill":True},
    style_kwds={"color":"blue"},
    map_kwds={"scrollWheelZoom":False},  
    control_scale=True,
    **kargs   
)


In [8]:
#Export de cette liste
stations_Synop.to_csv(r"C:\DATA\meteo\stations_Synop_sélectionnées.csv",index=False)
